# Getting Started with PsychoPy
[Psychopy](http://psychopy.org/) is an open-source application for running and presenting experimental stimuli. This tutorial aims to introduce users to the PsychoPy coder and IDE. In this tutorial we'll go over:
- Installing PsychoPy
- Using the 'Coder' IDE
- Coding a basic stroop task
- Saving and exporting response data

## Why PsychoPy? 
* It's open-source, so there is fairly extensive documentation and a user community to help solve provlems
* You're more familiar with Python or object-oriented programming
* It's free (no liscencing or dongles to lose)
* Flexible for different learning styles/levels of detail (good for programmers and non-programmers)

### Why *not* PsychoPy?
* Open-source means lots of versions and inevitable compatibility issues
* IDE is not conducive to in-line debugging
* You like your current presentation software/tools

# Resources
One of the things that drew me to PsychoPy (besides a lab that already used it) was the extensive documentation and tutorials that were already available! Below are a few resources that I quickly found via Google as well as those that I've used in the past.
- In general, I've found PsychoPy's [Discourse page](https://discourse.psychopy.org/) particiularly helpful

### Builder View ####
While this tutorial doesn't go over the Builder View, it's a GUI system that works well for quick learning and simple experimental designs. There's a LOT of documetation out there for help using the Builder View. Here are just a few:
- [From PsychoPy](http://www.psychopy.org/builder/builder.html)
- [And as a YouTube Video](https://www.youtube.com/watch?v=VV6qhuQgsiI)

### Coder View ####
- PsychoPy's [API](http://psychopy.org/api/api.html) is quite thorough, and good for going through all of the different settings for a given module
- This [tutorial](https://www.socsci.ru.nl/wilberth/nocms/psychopy/print.php) and this [workshop](https://lindeloev.net/psychopy-course/) that goes over some programming skills as well
- If reading chapters are your thing, [PsychoPy Is Fun!](https://link.springer.com/content/pdf/10.1007%2F978-3-319-64066-2_3.pdf)


# Installing PsychoPy
Because this tutorial uses a previous version of PsychoPy, if you want to follow along and not run into potential snags, be sure to download [version 1.90.3](https://github.com/psychopy/psychopy/releases/tag/1.90.3). PsychoPy has a number of ways to install the software, many of which are listed [here](http://psychopy.org/download.html#download) if you want to try and use the newer version. 

# The Coder View

PsychoPy's coder view is an rough-around-the-eduges IDE where you'll create and run your experiment scripts. There are a few important components:

![header](misc/psychopy1.png)

Here, the most important buttons you will use are the green 'go' button and the red 'stop' button. These will start and stop your script. Other useful tools on the header include the new/save files (first two buttons on the left) and the monitor center (the button with two monitors).

![footer](misc/psychopy2.png)

At the bottom of our IDE there are two important windows:
- The output window will display any information automatically returned from your script, as well as error information when your code breaks. When I'm developing a task, I'll often return objects that I need my script to create so that I can check that everything is working the way I think it does.
- The shell is a shell where you can run single lines of code. 

*Note*: If you want to run your experiment from the commandline, follow [these steps](https://github.com/lupyanlab/lab-computer/wiki/Install-psychopy-on-Anaconda-python)

# Setting Up Your Task

For this tutorial, we will build a very basic stroop task with the following conditions:
- Congruent/Incongruent
- 3 colors: Red, Blue, Green
- repeated 3 times for a total of 18 trials

The first thing we'll do is load in the python modules we'll be using:

In [1]:
from psychopy import data, event, visual, core
import os
import csv
import random

Next, we'll set up some parameters that we'll use later on in the experiment file.

In [2]:
# set our relative path
pwd = os.getcwd()

# timing parameters
display_time = 3
feedback_time = .5
fix_time = 1

timer = core.Clock()
globalClock = core.Clock()

# response keys
resp = ['1','2']

## Windows

To quote this [PsychoPy Tutorial](https://www.psychopy.org/coder/tutorial1.html):
> "Building stimuli is extremely easy. All you need to do is create a Window, then some stimuli. Draw those stimuli, then update the window."

It's important to note that PsychoPy was originally built for vision research, so some of the specific parameters for displaying visual information is beyond what we'll generally need.

In [3]:
# window setup
win = visual.Window([800,600], monitor='testMonitor', units='deg', fullscr=False,
                   allowGUI=False, screen=0)

win.close() # this is here bc i'm running psychopy in a notebook, you won't need this in the IDE

Now we'll add some additional stimuli. Can you see anything that seems to be missing from the stim?

In [4]:
# visual stim setup
colors = ['red','green','blue']

word = visual.TextStim(win, height=1, alignHoriz='center', alignVert='center')
fixation = visual.TextStim(win, text="+", height=2)

## The Trial Handler
The trial handler is how we're going to change trial-level data from trial to trial. This is probably the biggest benefit to PsychoPy. 

In [5]:
# setting up the trial handler

# first we'll read in our stim file
trial_data = [r for r in csv.DictReader(open('%s/misc/PsychoPy_Intro-stim.csv' % pwd, 'rU'))]

# we want 12 trials, so we'll repeat the process twice
# if you pre-randomize your data, you would change the method to 'sequential'
trials = data.TrialHandler(trial_data, nReps=3, method="random") 

## A brief sidebar on timing

If you noticed above, we initialized two separate 'clocks' to keep track of timing within our experiment. There are a lot of different ways to think about timing in your experiment, but I generally keep two separate clocks:
- a timer for shorter, within trial time management
- a 'global clock' for time management across a run/task

# The Task Loop

Let's write our task! Because each trial looks more or less the same, we'll essentially create a loop that takes the information from the trialHandler to create our trial structure. There are some important features to note:
- `trials.addData('variable name', variable)` adds a column of data to your log file and will record your variable of interest
- `word.setText('text')` changes the text for a given trial
- `timer.getTime()` will check the time of your clock object
- `event.getKeys()` allows you to log subject responses! you can time stamp them with this function or separately

I'd recommend checking out [the API](http://www.psychopy.org/api/api.html) for more info on what parameters each of these functions requires.

In [ ]:
# this will be the FIRST AND ONLY TIME YOU RESET THE GLOBAL CLOCK
globalClock.reset()

# create a task start variable so we can calculate RT
task_onset = globalClock.getTime()

for trial in trials:
    # let's clear any events that were stored from the previous trial
    event.clearEvents()
    
    # first we need to set the text!
    word.setText(trial['color'])
    
    # now we'll set the color 
    if trial['congruent'] == 'Y':
        color = trial['color']
        word.setColor(color)
    else:
        # let's randomly select an 'incongruent' color
        random.shuffle(colors)
        
        # if they shuffle accidentally matches, we'll take the next one
        if colors[0] == trial['color']:
            color = colors[1]
        else:
            color = colors[0]
            
        word.setColor(color)
    
    # let's record in our data what 'color' was selected
    trials.addData('text_color', color)
    
    # let's show people some data and wait for their response!
    
    # always be sure to reset your timer!
    
    timer.reset()
    
    while timer.getTime() < display_time:
        word.draw()
        win.flip()
        response = event.getKeys(keyList = resp, timeStamped = globalClock)
        
        # if there's a response, let's keep the screen online for .5 more seconds
        if len(response) > 0: 
            resp_val = int(response[0][0])
            rt_onset = response[0][1]
            rt = rt_onset - task_onset
            
            core.wait(feedback_time)
            break
        
        else: #if the subject doesn't respond
            resp_val = 'NA'
            rt_onset = 'NA'
            rt = 'NA'
            
    
    # now let's add record the data we collected
    trials.addData('resp', resp_val)
    trials.addData('rt', rt)
    
    # let's display a fixation between trials
    
    # first we have to reset the timer
    timer.reset()
    
    while timer.getTime() < fix_time:
        fixation.draw()
        win.flip()

## let's save our data
The last thing we'll do is write out or data. Because this is a toy example, we're just going to save a file with a different name than the stim file, but in the future you would want to save it to a subject log file.

In [ ]:
# let's save out the data
trials.saveAsWideText('%s/misc/task_data.csv' % pwd, delim = ',', appendFile=True)

# Thanks!
If you're interested in learning more about PsychoPy, don't hestitate to reach out! Also, there are task examples that have been used to collect data on [my GitHub](https://github.com/elizabethbeard/tasks) if you'd like to see some of the additional steps to collect subject-level data.